### 01. 크롤링 

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import html5lib
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException
import numpy as np
import pandas as pd

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('headless') # 웹브라우저를 띄우지 않는 headless 옵션
options.add_argument('disable-gpu')    # GPU 사용 안함
options.add_argument('lang=ko_KR')    # 언어 설정, options=options
driver = webdriver.Chrome('chromedriver', options=options)

In [87]:
movie_list = pd.read_excel('movie_list.xlsx', parse_dates=['제작연도'])
on = movie_list[movie_list['제작상태'] == '개봉']

In [88]:
#on

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,레드 아워,Red Hour,2021-01-01,미국,장편,"액션,스릴러",개봉,NaN,NaN
1,제불찰씨 이야기,The Story Of Mr. Sorry,2008-01-01,한국,장편,"애니메이션,드라마,스릴러,미스터리",개봉,"류지나,이혜영,김일현,곽인근,이은미",한국영화아카데미
2,천년여우,Millennium Actress,2001-01-01,일본,장편,"스릴러,애니메이션",개봉,곤 사토시,매드하우스(マッドハウス)
3,아브릴과 조작된 세계,April and the Extraordinary World,2015-01-01,"프랑스,벨기에,캐나다",장편,"애니메이션,어드벤처,코미디",개봉,"크리스티앙 데마르,프랑크 에킨시",NaN
4,극장판 귀멸의 칼날: 무한열차편,Demon Slayer: Kimetsu no Yaiba the Movie - Mug...,2020-01-01,일본,장편,애니메이션,개봉,소토자키 하루오,NaN
...,...,...,...,...,...,...,...,...,...
18988,천방지축,Cat And Mouse,2003-01-01,홍콩,장편,"액션,코미디",개봉,진가상,NaN
18989,콜드 크릭,Cold Creek Manor,2003-01-01,미국,장편,스릴러,개봉,마이크 피기스,NaN
18990,터미널,The Terminal,2004-01-01,미국,장편,드라마,개봉,스티븐 스필버그,NaN
18991,프락치,Spying Cam,2004-01-01,한국,장편,드라마,개봉,황철민,씨네굿


In [24]:
for i in range(0, len(on), 200):
    movie_list_1 = []
    movie_list_1 = on[i:i+200]
    movie_list_1.to_csv('movie_%d.csv' %i,index = False)

In [8]:
for j in range(18200, 18400, 200) : 
    name = pd.read_csv("./movie_%d.csv" %j)
    movie = name["영화명"]
    
    driver = webdriver.Chrome('C:/Users/realy/Desktop/2021-2/crawling/chromedriver.exe')
    driver.get("https://pedia.watcha.com/ko-KR")
    driver.implicitly_wait(4)
    result ={}

    for title in tqdm(movie):
        #제목 검색
        elem = driver.find_element_by_name('searchKeyword')
        elem.send_keys(title)
        elem.send_keys(Keys.RETURN)
        driver.implicitly_wait(3)
        
        try:
            review_list = []
            #영화선택
            driver.find_element_by_xpath('//*[@id="root"]/div/div[1]/section/section/div[3]/div[1]/section/section[2]/div[2]/div[1]/div/ul/li[1]/a/div[2]/div[1]/div[1]').click()
            #더보기
            Url=driver.current_url+"/comments"
            driver.get(Url)
            time.sleep(0.5)
            #스크롤 내리기
            driver.execute_script("window.scrollTo(0, 2000)")
            time.sleep(0.5)
            driver.execute_script("window.scrollTo(2000, 4000)")
            time.sleep(0.5)
            driver.execute_script("window.scrollTo(4000, 6000)")
            time.sleep(0.5)
            #코멘트 읽기
            x = driver.find_elements_by_xpath('//*[@id="root"]/div/div[1]/section/section/div/div/div/ul/div')
        
            # 코멘트별 저장 
            for i in range(0,len(x)-2) :
                comment = driver.find_element_by_css_selector('#root > div > div.css-1fgu4u8 > section > section > div > div > div > ul > div:nth-child(%d) > div.css-4tkoly > div > span'%(i+1)).text
                comment = comment.replace('\n','')
                comment = comment.replace('#','')
                review_list.append(comment)

            result[title] = review_list
            driver.get("https://pedia.watcha.com/ko-KR")
            
        except NoSuchElementException:
            driver.get("https://pedia.watcha.com/ko-KR")
    
    df = pd.DataFrame(result.items(), columns = ['movie_id' , 'reviews'])
    df.to_csv('./review%d.csv'%(j), index =False, encoding="UTF-8-sig")  

  0%|          | 0/179 [00:00<?, ?it/s]